여기서는 이미지를 openCV를 활용하여 자르고, 정확한 bbox를 구한다.</br>
그 뒤, 각 이미지마다 bbox 기준으로 자른 후 224x224 크기의 투명 이미지의 중앙에 합성하여 분류기용 이미지로 만든다.</br>
분류기용 이미지들을 해당하는 폴더들로 수작업으로 분류해준다.</br>
</br>
분류가 완료되면 2가지 작업을 수행한다.

- 분류기에 넣을 dataset으로 만들어서 사전학습된 분류기를 학습시킨다.
- 이미 주어진 annotation 파일을 활용하여 bbox 값을 수정하거나, 라벨링이 없는 데이터들에 대해 bbox를 추가하여 annotation을 만들어준다.

여기서는 2가지 가정을 한다.

- 학습 데이터로 주어지는 이미지에서, 모든 알약은 충분한 간격을 두고 떨어져 있으며, 가로로 두 구역으로 나눌 때 두 구역 사이에는 충분한 간격이 있다.
- 이 파일에서 사용하는 모든 이미지는 배경이 제거되어 있는 것을 사용한다.

In [ ]:
import os
import json
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import operator
import pandas as pd
import copy

# 이미지 자르기
가장 먼저, 각 이미지마다 3~4개의 이미지로 나눠준다.</br>
자른 뒤, 원본 이미지의 크기와 각 이미지가 원본에서 어느 범위에 해당하는지를 텍스트 파일로 저장한다.</br>
아래의 변환 함수는 [링크](https://dreamfuture.tistory.com/50)의 블로그 글을 차용하였다.

In [ ]:
def PIL2OpenCV(pil_image):
    numpy_image= np.array(pil_image)
    opencv_image = cv2.cvtColor(numpy_image, cv2.COLOR_RGB2BGR)
    return opencv_image

def OpenCV2PIL(opencv_image):
    color_coverted = cv2.cvtColor(opencv_image, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(color_coverted)
    return pil_image

In [ ]:
def image_cutter(images_path='./ai06-level1-project/train_output',annotations_path='./ai06-level1-project/train_annotations'):
    images=sorted(os.listdir(images_path))
    annotations=sorted(os.listdir(annotations_path))

    count=0
    for image in images:
        count+=1
        print(count)
        # 알약 조합은 이름에서 확장자 및 이미지 정보를 제거한 것과 같다.
        name=image[:-23]

        # 만약 annotation에 해당 알약 조합의 정보가 없다면, 폴더를 만들어준다.
        if not os.path.isdir(annotations_path,name+"_json"):
            os.makedirs(annotations_path,name+"_json")

        # 이미지 파일을 불러온다.
        image_file=Image.open(os.path.join(images_path,image))

        # 이미지 파일을 cv 파일로, 배경을 유지하여 만들어준다.
        image_cv=cv2.cvtColor(np.array(image_file), cv2.COLOR_RGBA2BGRA)

        # 이미지 파일을 cv 파일로, 배경을 지워서 만들어준다.
        image_file=PIL2OpenCV(image_file.convert("RGB"))

        # 이미지 파일에 변형을 가할 파일을 만들어준다.
        conv_file=np.zeros_like(image_file)

        # 이미지 파일 원본의 크기를 구한다.
        H,W,C=image_file.shape

        # 각 칸에 대해, 배경에 해당하면 흰색, 아니면 검은 색으로 만들어준다.
        conv_file[(image_file==[130,130,130]).all(axis=2)]=255
        
        # 검출된 3~4개의 이미지를 저장할 배열이다.
        imgs=[]

        # 행 단위로 (horizontal) 직전 행의 모든 칸이 흰색인지 체크한다.
        check_h=True
        h_line=[]
        for h in range(H):
            # 이번 행에서 모든 칸이 흰색인가에 대해서
            cur_row=(conv_file[h]==255).all()
            # 만약 cur_row와 check_h가 서로 다르다면 h_line에 추가
            if check_h != cur_row:
                if len(h_line)>0:
                    if h_line[-1]+50>h:
                        continue
                h_line.append(h)
                check_h=cur_row
        # 객체가 테두리에 맞닿아 있거나, 테두리에 의해 잘린 경우가 있다.
        if len(h_line)%2==1:
            h_line.append(H-1)
        # 행 단위로 나누는 선을 만든다.
        h_line=[h_line[0]//2,(h_line[1]+h_line[2])//2,((h_line[3]+H)//2)]

        # 행 단위로 2개의 파트를 만든다.
        conv1=conv_file[h_line[0]+1:h_line[1]+1]
        conv2=conv_file[h_line[1]+1:h_line[2]+1]

        # 파트 1의 이미지 수를 구한다.
        check_v=True
        v_line1=[]
        for w in range(W):
            # 이번 열에서 파트 1의 모든 칸이 흰색인가에 대해서
            cur_col=(conv1[:,w]==255).all()
            # 만약 cur_col과 check_v가 서로 다르다면 v_line1에 추가
            if check_v != cur_col:
                if len(v_line1)>0:
                    if v_line1[-1]+50>w:
                        continue
                v_line1.append(w)
                check_v=cur_col
        # 객체가 테두리에 맞닿아 있거나, 테두리에 의해 잘린 경우가 있다.
        if len(v_line1)%2==1:
            v_line1.append(W-1)
        # v_line1의 길이가 4라면, imgs에 2개의 이미지를 넣어준다.
        if len(v_line1)==4:
            v_line1=[v_line1[0]//2,(v_line1[1]+v_line1[2])//2,((v_line1[3]+W)//2)]
            imgs.append(image_cv[h_line[0]:h_line[1],v_line1[0]:v_line1[1]])
            imgs.append(image_cv[h_line[0]:h_line[1],v_line1[1]:v_line1[2]])
        # 아니라면 imgs에는 1개의 이미지가 추가된다.
        else:
            v_center=sum(v_line1)//2
            v_line1=[max(0,v_center-256),min(W,v_center+256)]
            imgs.append(image_cv[h_line[0]:h_line[1],v_line1[0]:v_line1[1]])

        # 파트 2에 대해서도 마찬가지로 한다.
        check_v2=True
        v_line2=[]
        for w in range(W):
            cur_col2=(conv2[:,w]==255).all()
            # 만약 cur_col과 check_v가 서로 다르다면 v_line2에 추가
            if check_v2 != cur_col2:
                if len(v_line2)>0:
                    if v_line2[-1]+50>w:
                        continue
                v_line2.append(w)
                check_v2=cur_col2
        # 객체가 테두리에 맞닿아 있거나, 테두리에 의해 잘린 경우가 있다.
        if len(v_line2)%2==1:
            v_line2.append(W-1)
        # v_line2의 길이가 4라면, imgs에 2개의 이미지를 넣어준다.
        if len(v_line2)==4:
            v_line2=[v_line2[0]//2,(v_line2[1]+v_line2[2])//2,((v_line2[3]+W)//2)]
            imgs.append(image_cv[h_line[1]:h_line[2],v_line2[0]:v_line2[1]])
            imgs.append(image_cv[h_line[1]:h_line[2],v_line2[1]:v_line2[2]])
        # 아니라면 imgs에는 1개의 이미지가 추가된다.
        else:
            v_center=sum(v_line2)//2
            v_line2=[max(0,v_center-256),min(W,v_center+256)]
            imgs.append(image_cv[h_line[1]:h_line[2],v_line2[0]:v_line2[1]])

        # imgs의 각 이미지를 폴더에 저장해준다.
        for idx,img in enumerate(imgs):
            image_i=Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGRA2RGBA))
            image_i.save(f"./cropped_images/{image[:-4]}_{str(idx+1)}.png", format="PNG")

        # 이미지 정보를 txt로 저장한다.
        with open(f'{annotations_path}/{name}_json/{image[:-4]}.txt','w') as f:
            f.write(f'H: {H}\n')
            f.write(f'W: {W}\n')
            cnt=0
            # 만약 파트 1의 이미지가 2개라면
            if len(v_line1)==3:
                f.write(f'img{str(cnt+1)}: {h_line[0]},{h_line[1]},{v_line1[0]},{v_line1[1]}\n')
                f.write(f'img{str(cnt+2)}: {h_line[0]},{h_line[1]},{v_line1[1]},{v_line1[2]}\n')
                cnt+=2
            else:
                f.write(f'img{str(cnt+1)}: {h_line[0]},{h_line[1]},{v_line1[0]},{v_line1[1]}\n')
                cnt+=1
            # 만약 파트 2의 이미지가 2개라면
            if len(v_line2)==3:
                f.write(f'img{str(cnt+1)}: {h_line[1]},{h_line[2]},{v_line2[0]},{v_line2[1]}\n')
                f.write(f'img{str(cnt+2)}: {h_line[1]},{h_line[2]},{v_line2[1]},{v_line2[2]}\n')
                cnt+=2
            else:
                f.write(f'img{str(cnt+1)}: {h_line[1]},{h_line[2]},{v_line2[0]},{v_line2[1]}\n')
                cnt+=1

image_cutter(images_path='./ai06-level1-project/train_output',annotations_path='./ai06-level1-project/train_annotations')

이로써, 각 annotation 폴더는 하나의 '알약 조합'에 대한 여러 이미지들의 설명 파일을 갖게 된다.</br>
각 설명 파일의 처음 2줄은 원본 이미지의 크기이다.</br>
이후, 3~4개의 세부 이미지에 대해 그 이미지들의 원본(전체)이미지에서의 위치를 x1,x2,y1,y2로 갖는다.

# 상세 bbox 구하기
이제 각 상세 이미지마다 bbox를 구한다.</br>
이 과정에서의 bbox 정보는 동일한 위치에, 새로운 txt 파일로 저장한다.

In [ ]:
def bbox_detect(annotation_root='./ai06-level1-project/train_annotations', image_root='./ai06-level1-project/train_output'):
    # 각 이미지 정보 폴더를 순회한다.
    annotations=sorted(os.listdir(annotation_root))

    cnt=0

    for annotation in annotations:
        # txt 파일들 목록을 불러온다.
        image_info=copy.deepcopy([entry.name for entry in os.scandir(os.path.join(annotation_root,annotation)) if entry.is_file() and entry.name.endswith(".txt")])
        
        # 이미지 하나에 대한 txt 파일마다
        for sub_info in image_info:
            if 'bbox' in sub_info:
                continue
            print(cnt+1)
            cnt+=1
            
            image_name=sub_info[:-4]
            # 이미지 이름은 sub_info에서 마지막 확장자를 지운 것과 같다.
            image=Image.open(os.path.join(image_root,image_name+'.png'))

            # 이미지 파일을 cv 파일로, 배경을 유지하여 만들어준다.
            image_cv=cv2.cvtColor(np.array(image), cv2.COLOR_RGBA2BGRA)

            # 이미지 파일을 cv 파일로, 배경을 지워서 만들어준다.
            image_file=PIL2OpenCV(image.convert("RGB"))

            # 이미지 파일에 변형을 가할 파일을 만들어준다.
            conv_file=np.zeros_like(image_file)

            # 이미지 파일 원본의 크기를 구한다.
            H,W,C=image_file.shape

            # 각 칸에 대해, 배경에 해당하면 흰색, 아니면 검은 색으로 만들어준다.
            conv_file[(image_file==[130,130,130]).all(axis=2)]=255

            with open(os.path.join(annotation_root,annotation,sub_info),'r') as f:
                # 이미지의 높이, 너비 정보를 얻는다.
                H=f.readline().strip()[3:]
                W=f.readline().strip()[3:]
                parts=f.readlines()

                h_line=[]
                v_line=[]
                
                for part in parts:
                    part=part.strip().split()
                    part_num=part[0][3:-1]
                    y1,y2,x1,x2=map(int,part[1].split(','))

                    # 현재 부분 이미지에서 수평선 기준으로 상하 경계선을 구한다.
                    check=True
                    # 최외곽 픽셀을 기준으로 경계선을 구하기 위함이다.
                    h_dot=[]
                    for y in range(y1,y2):
                        cur_check=(conv_file[y,x1:x2]==255).all()
                        if check != cur_check:
                            h_dot.append(y)
                        check=cur_check
                        if len(h_dot) == 1:
                            h_dot.append(x2)

                    # 현재 부분 이미지에서 수직선 기준으로 좌우 경계선을 구한다.
                    check=True
                    # 최외곽 픽셀을 기준으로 경계선을 구하기 위함이다.
                    v_dot=[]
                    for x in range(x1,x2):
                        cur_check=(conv_file[y1:y2,x]==255).all()
                        if check != cur_check:
                            v_dot.append(x)
                        check=cur_check
                        if len(v_dot) == 1:
                            v_dot.append(y2)
                        
                    # 앞서 구한 픽셀들 중 첫 픽셀과 마지막 픽셀을 활용한다.
                    h_line.append(h_dot[0])
                    h_line.append(h_dot[-1])
                    v_line.append(v_dot[0])
                    v_line.append(v_dot[-1])
                
                # 전체 부분 이미지에 대해 bbox를 저장할 파일을 연다.
                with open(os.path.join(annotation_root,annotation,image_name+'_bbox.txt'),'w') as f:
                    f.write('id\ty1\ty2\tx1\tx2\n')
                    # 각 부분 이미지마다 세부 bbox를 저장한다.
                    for p in range(len(parts)):
                        f.write(f'img{str(p+1)}: {h_line[p*2]},{h_line[p*2+1]},{v_line[p*2]},{v_line[p*2+1]}\n')

bbox_detect(annotation_root='./ai06-level1-project/train_annotations', image_root='./ai06-level1-project/train_output')

# 이미지 저장 및 라벨링하기
이제 각 객체 단위로 이미지를 자른 후, 빈 배경의 256 x 256 이미지의 중앙에 합성한다.

In [ ]:
def image_crop_with_bbox(annotation_root='./ai06-level1-project/train_annotations',image_root='./ai06-level1-project/train_output',crop_path='./cropped_images/'):
    # 각 이미지 정보 폴더를 순회한다.

    annotations=sorted(os.listdir(annotation_root))

    cnt=0

    for annotation in annotations:
        # txt 파일들 목록을 불러온다.
        image_info=copy.deepcopy([entry.name for entry in os.scandir(os.path.join(annotation_root,annotation)) if entry.is_file() and entry.name.endswith(".txt")])
        
        # 이미지 하나에 대한 txt 파일마다
        for sub_info in image_info:
            if 'bbox' not in sub_info:
                continue
            print(cnt+1)
            cnt+=1
            
            image_name=sub_info[:-4]
            # 이미지 이름은 sub_info에서 마지막 확장자를 지운 것과 같다.
            image=Image.open(os.path.join(image_root,image_name[:-5]+'.png'))

            # 이미지 파일을 cv 파일로, 배경을 유지하여 만들어준다.
            image_cv=cv2.cvtColor(np.array(image), cv2.COLOR_RGBA2BGRA)

            # 이미지 파일을 cv 파일로, 배경을 지워서 만들어준다.
            image_file=PIL2OpenCV(image.convert("RGB"))

            # 이미지 파일 원본의 크기를 구한다.
            H,W,C=image_file.shape

            with open(os.path.join(annotation_root,annotation,sub_info),'r') as f:
                f.readline()
                # 이미지의 높이, 너비 정보를 얻는다.
                parts=f.readlines()
                
                # 각 세부 이미지마다
                for part in parts:
                    part=part.strip().split()
                    # id와 좌표를 얻는다.
                    part_id=part[0][3:-1]
                    y1,y2,x1,x2=map(int,part[1].split(','))
                    # 배경을 유지한 이미지에서 세부 이미지를 잘라낸다.
                    img=image_cv[y1:y2,x1:x2]
                    img=Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGRA2RGBA))
                    
                    # 크기 640x640의 투명 배경인 빈 이미지를 만든다.
                    canvas_size = (640,640)
                    canvas = Image.new("RGBA", canvas_size, (0,0,0,0))

                    h,w=y2-y1,x2-x1
                    x=(640-w)//2
                    y=(640-h)//2
                    canvas.paste(img,(x,y),img)
                    canvas.save(os.path.join(crop_path,image_name[:-4]+part_id+'.png'))

image_crop_with_bbox(annotation_root='./ai06-level1-project/train_annotations',image_root='./ai06-level1-project/train_output',crop_path='./cropped_images/')

테스트용 코드입니다.</br>
적절히 좌표 및 이미지 이름을 넣어서 사용하세요.</br>
상자 이미지가 잘 보이도록 출력물에서 알파 채널을 지웠습니다.

In [ ]:
image_root='./ai06-level1-project/train_output'
image_name='K-001900-010224-016551-031705_0_2_0_2_70_000_200'
image=Image.open(os.path.join(image_root,image_name+'.png'))
image_cv=cv2.cvtColor(np.array(image), cv2.COLOR_RGBA2BGR)
pt1 = (154, 145)
pt2 = (368, 535)
cv2.rectangle(image_cv, pt1, pt2, (0, 256, 0), 2)
pt3 = (597, 144)
pt4 = (828, 573)
cv2.rectangle(image_cv, pt3, pt4, (0, 256, 0), 2)
pt5 = (185, 771)
pt6 = (379, 1154)
cv2.rectangle(image_cv, pt5, pt6, (0, 256, 0), 2)
pt7 = (657, 866)
pt8 = (846, 1010)
cv2.rectangle(image_cv, pt7, pt8, (0, 256, 0), 2)

fig, ax = plt.subplots(1, figsize=(12,9))
plt.imshow(cv2.cvtColor(image_cv, cv2.COLOR_RGB2BGR))
plt.show()

In [ ]:
max_width,max_height

# 데이터 라벨링 검토하기
우선 라벨링 파일을 호출하고, 이미지 단위로 라벨링 매핑 dictionary를 생성한다.</br>
그와 동시에, 클래스 id 별로 이미지 폴더를 만들어서 저장해준다.</br>
</br>
완료된 후, 잘린 이미지 중 라벨링 매핑의 key에 없는 것을 찾아서 추가로 매핑한다.</br>
또한, 클래스 id 별로 이미지 폴더를 검토한다.

In [ ]:
# 잘린 이미지들이 위치한 폴더
image_root='./cropped_images'
class_id=0
labelling_map={}
class_count={}
# 라벨링 파일
with open('./label_mapping.txt','r') as f:
    # 첫 줄 무시
    f.readline()
    lines=f.readlines()
    for line in lines:
        # 이름과 라벨 사이에는 탭으로 구분했습니다.
        line=list(line.strip().split('\t'))
        print(line[0])
        # 라벨이 있다면 class_id를 갱신
        if len(line)==2:
            class_id=int(line[1])
            class_count[str(class_id)]=0
        # 현재 이미지의 라벨은 현재 class_id
        labelling_map[line[0]]=class_id
        class_count[str(class_id)]+=1
        # 현재 class_id에 해당하는 폴더가 없으면 생성
        os.makedirs(os.path.join('pill_images',str(class_id)),exist_ok=True)
        # 이미지를 해당 폴더로 복사
        os.system(f'cp {image_root}/{line[0]}.png ./pill_images/{str(class_id)}/{line[0]}.png')

이제 이미지 중 놓친 것이 있는지 확인한다.

In [ ]:
images=os.listdir(image_root)
for image in images:
    if image[:-4] not in labelling_map.keys():
        print(image[:-4])

모든 파일에 대해 라벨링 및 검토를 마무리하였다.</br>
또한, 중간에 잘못 이름이 붙은 파일에 대해서도 수정을 해주었다.</br>
</br>
이제 각 클래스 별 이미지 수를 통해 불균형 여부를 파악한다.

In [ ]:
class_count_df=pd.DataFrame.from_dict(data=class_count,orient='index')
class_count_df.plot(kind='bar')

대충 봐도 불균형이 심각하니, 데이터를 추가하거나 증강을 할 필요가 있다.</br>
일단 여기서 일시 중단하고, 추가 데이터를 얻어서 위의 과정을 반복하겠다.</br>
그리고, 각 클래스 별로 종합적인 데이터 수를 구하고, 데이터 수가 가장 적은 클래스를 활용할 것이다.</br>
이 과정은 해당 시점에서 다시 설명하겠다.

# bbox 데이터를 annotation화하기
앞서 획득한 데이터는 YOLO 데이터로도 변환이 가능하다.</br>
이는 다음과 같이 이뤄진다.

- 각 원본 이미지에 대해 (배경을 지운 버전)
    - 해당 이미지의 각 bbox 데이터에서 '객체 별 id'와 'bbox 정보'를 얻는다.
    - '객체 별 id'를 사용하여, 라벨링 데이터에서 '해당 객체의 class id'를 얻는다. ('이미지 이름'_'객체 id')와 'class id'가 매핑되고 있기에 가능하다.
    - 이미지에 대한 id는 임의로 1번부터 시작한다.
    - annotation에 대한 id는 임의로 1번부터 시작한다.
    - 각 bbox에 대해
        - annotation id와 bbox를 annotation 데이터에 추가한다.
        - class id를 class 정보로 추가한다. supercategory는 전부 pill로 통일한다. (일단은)

위의 방법을 활용하여 다음과 같이 실행한다.

In [ ]:
cat_id_to_name={
    1899: "보령부스파정 5mg",
    2482: "뮤테란캡슐 100mg",
    3350: "일양하이트린정 2mg",
    3482: "기넥신에프정(은행엽엑스)(수출용)",
    3543: "무코스타정(레바미피드)(비매품)",
    3742: "알드린정",
    3831: "뉴로메드정(옥시라세탐)",
    4377: "타이레놀정500mg",
    4542: "에어탈정(아세클로페낙)",
    5093: "삼남건조수산화알루미늄겔정",
    5885: "타이레놀이알서방정(아세트아미노펜)(수출용)",
    6191: "삐콤씨에프정 618.6mg/병",
    6562: "조인스정 200mg",
    10220: "쎄로켈정 100mg",
    10223: "넥시움정 40mg",
    12080: "리렉스펜정 300mg/PTP",
    12246: "아빌리파이정 10mg",
    12419: "자이프렉사정 2.5mg",
    12777: "다보타민큐정 10mg/병",
    13394: "써스펜8시간이알서방정 650mg",
    13899: "에빅사정(메만틴염산염)(비매품)",
    16231: "리피토정 20mg",
    16261: "크레스토정 20mg",
    16547: "가바토파정 100mg",
    16550: "동아가바펜틴정 800mg",
    16687: "오마코연질캡슐(오메가-3-산에틸에스테르90)",
    18109: "란스톤엘에프디티정 30mg",
    18146: "리리카캡슐 150mg",
    18356: "종근당글리아티린연질캡슐(콜린알포세레이트)",
    19231: "콜리네이트연질캡슐 400mg",
    19551: "트루비타정 60mg/병",
    19606: "스토가정 10mg",
    19860: "노바스크정 5mg",
    20013: "마도파정",
    20237: "플라빅스정 75mg",
    20876: "엑스포지정 5/160mg",
    21025: "펠루비정(펠루비프로펜)",
    21324: "아토르바정 10mg",
    21770: "라비에트정 20mg",
    22073: "리피로우정 20mg",
    22346: "자누비아정 50mg",
    22361: "맥시부펜이알정 300mg",
    22626: "메가파워정 90mg/병",
    23202: "쿠에타핀정 25mg",
    23222: "비타비백정 100mg/병",
    24849: "놀텍정 10mg",
    25366: "자누메트정 50/850mg",
    25437: "큐시드정 31.5mg/PTP",
    25468: "아모잘탄정 5/100mg",
    27652: "세비카정 10/40mg",
    27732: "트윈스타정 40/5mg",
    27776: "카나브정 60mg",
    27925: "울트라셋이알서방정",
    27992: "졸로푸트정 100mg",
    28762: "트라젠타정(리나글립틴)",
    29344: "비모보정 500/20mg",
    29450: "레일라정",
    29666: "리바로정 4mg",
    29870: "렉사프로정 15mg",
    30307: "트라젠타듀오정 2.5/850mg",
    31704: "낙소졸정 500/20mg",
    31862: "아질렉트정(라사길린메실산염)",
    31884: "자누메트엑스알서방정 100/1000mg",
    32309: "글리아타민연질캡슐",
    33008: "신바로정",
    33207: "에스원엠프정 20mg",
    33877: "브린텔릭스정 20mg",
    33879: "글리틴정(콜린알포세레이트)",
    34596: "제미메트서방정 50/1000mg",
    35205: "아토젯정 10/40mg",
    36636: "로수젯정10/5밀리그램",
    38161: "로수바미브정 10/20mg",
    41767: "카발린캡슐 25mg",
    44198: "케이캡정 50mg"
}

In [ ]:
def parse_bbox_txt(txt_path,an_id,img_id,img_name):
    annotations = []
    categories = []
    with open(txt_path, "r", encoding="utf-8") as f:
        # 첫 줄 제거 - 이전 단계의 파일 형식에서 첫 줄은 인덱스
        f.readline()
        lines = f.read().strip().split("\n")
    a_id=an_id
    for idx, line in enumerate(lines):
        if ":" not in line:
            continue

        _, coords = line.split(":")
        y1, y2, x1, x2 = map(int, coords.split(","))
        w = x2 - x1
        h = y2 - y1

        cat_id=labelling_map[img_name[:-4]+'_'+str(idx+1)]

        annotations.append({
            "id": a_id,
            "image_id": img_id,
            "category_id": cat_id,
            "bbox": [x1, y1, w, h],
            "area": w * h,
            "iscrowd": 0,
            "segmentation": []
        })
        a_id+=1

        categories.append({
            "id": cat_id,
            "supercategory": 'pill',
            "name": cat_id_to_name[cat_id]
        })
    return annotations, categories, a_id

In [ ]:
def bbox_to_json(images_root='./ai06-level1-project/train_output',annotations_root='./ai06-level1-project/train_annotations'):
    pill_sets=os.listdir(annotations_root)
    annotation_id=1
    image_id=1
    cnt=0
    for pill_set in pill_sets:
        pill_set_path = os.path.join(annotations_root, pill_set)

        # *_bbox.txt 파일만 가져오기
        bbox_files = [f for f in os.listdir(pill_set_path) if f.endswith("_bbox.txt")]

        if len(bbox_files) == 0:
            continue
        
        new_json_dir = os.path.join(pill_set_path)

        # 각 이미지에 대해서
        for bbox_file in bbox_files:
            txt_path = os.path.join(pill_set_path, bbox_file)
            base_name = bbox_file[:-9]
            image_file_name = base_name + ".png"
            json_filename = base_name + ".json"
            json_path = os.path.join(new_json_dir, json_filename)

            # 이미지 크기를 구하기 위해 호출
            image=Image.open(os.path.join(images_root,image_file_name))
            W,H=image.size
            
            annotations,categories, a_id=parse_bbox_txt(txt_path,annotation_id,image_id,image_file_name)
            annotation_id=a_id

            coco_dict = {
                "images": [
                    {
                        "id": image_id,
                        "file_name": image_file_name,
                        "height": H,
                        "width": W
                    }
                ],
                "annotations": annotations,
                "categories": categories
            }

            image_id+=1
            with open(json_path, "w", encoding="utf-8") as f:
                json.dump(coco_dict, f, ensure_ascii=False, indent=4)

            cnt += 1
            print(cnt)

bbox_to_json(annotations_root='./ai06-level1-project/train_annotations')

# 분류기 학습 및 예측하기
각 이미지 조각에 대해, 분류기를 통해 학습을 시킨다.</br>
학습이 완료되면, 앞서 train 데이터에 적용한 방법을 사용하여 test 데이터를 생성한다.</br>
분류기를 통해, test 데이터의 이미지들에 대해 분류를 수행한다.

In [ ]:
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
import torch.nn as nn

num_classes = 56

이제 데이터셋을 준비한다.

In [ ]:
annotation_root='./ai06-level1-project/train_annotations'
annotations=sorted(os.listdir(annotation_root))

image_root='./ai06-level1-project/train_output'

code=[]
images = os.listdir(image_root)
for i in images:
    i=i[2:-23]
    codes=list(map(int,i.split('-')))
    code.extend(codes)

code=sorted(list(set(code)))
code

In [ ]:
len(code)

그리고, 모델을 준비해준다.

In [ ]:
# 백본만 사전학습된 상태로 로드
model = efficientnet_b0(weights=EfficientNet_B0_Weights.IMAGENET1K_V1)

# 분류기 헤드 교체
in_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(in_features, num_classes)

# YOLO 학습하기
앞서 만들었던 bbox 데이터를 활용하여 YOLO 학습용 annotation 파일들을 만든다.</br>
그 파일들을 이용하여 YOLO 학습용 dataset을 만든 뒤 적용한다.